In [1]:
import numpy as np
import statsmodels
from causalimpact import CausalImpact
import pandas as pd
from scipy import stats
import matplotlib
import seaborn as sns
import lifelines





In [2]:
dfc = pd.DataFrame(pd.read_pickle('compliance.pkl'))

In [3]:
dfb = pd.DataFrame(pd.read_pickle('survey.pkl'))
tiers = pd.DataFrame(pd.read_csv('tiers.csv'))


In [4]:
#tiers = tiers.drop('Unnamed: 0',axis = 1)
tiers = tiers.dropna()

dfb = dfb.merge(tiers,right_on = 'Fitbit',left_on = 'id')


In [5]:
df = pd.DataFrame(pd.read_csv('compliance_data.csv'))

In [6]:
dfb.head()

,id,stepChallenge,gender_1,race_1,parentincome_1,Extraversion_1,Extraversion_2,Extraversion_6,Agreeableness_1,Agreeableness_2,...,Conscientiousness_6,Openness_1,Openness_2,Openness_6,Neuroticism_1,Neuroticism_2,Neuroticism_6,Unnamed: 0,Tier,Fitbit
0,3M4MLH,1.0,2.0,1.0,1.0,3.750,3.750,3.0,4.000000,3.555556,...,2.666667,2.6,3.000000,2.555556,3.375,3.25,3.125,0,Tier1,3M4MLH
1,3NVWH4,0.0,2.0,2.0,5.0,3.875,4.250,NaN,3.000000,2.666667,...,NaN,3.0,3.555556,NaN,3.750,4.00,NaN,2,Tier1,3NVWH4
2,3YFW8D,1.0,2.0,5.0,3.0,2.875,NaN,NaN,3.000000,NaN,...,NaN,3.2,NaN,NaN,3.125,NaN,NaN,6,Tier3,3YFW8D
3,4RNCLT,1.0,1.0,2.0,7.0,3.500,NaN,NaN,3.555556,NaN,...,NaN,3.3,NaN,NaN,3.875,NaN,NaN,8,Tier3,4RNCLT
4,3TPKGY,0.0,2.0,2.0,3.0,4.000,3.625,NaN,2.888889,3.333333,...,NaN,3.8,4.555555,NaN,3.250,3.25,NaN,10,Tier2,3TPKGY


In [7]:
s1Start = " 2017-01-12"
s1End = '2017-04-27'
s2Start = "2017-04-28"
s2End = '2018-04-22'

In [8]:
df = df[['participID','date','week','complyPercent']]

In [9]:
df["Spring"] = 0
df.loc[((df.date >= s1Start)&(df.date <= s1End )) , 'Spring'] = 1
df.loc[((df.date >= s1Start)) , 'Start'] = 1
df = df[df['Start'] == 1]

dfF = df[df['Spring'] == 1]
groupedF =dfF.groupby('participID')

df['week_after_intervention'] = ((pd.to_datetime(df['date']) - pd.to_datetime(s2Start)).dt.days / 7)

In [10]:
grouped =df.groupby('participID')
grouped = grouped['complyPercent'].mean()
participI  =  grouped.index
mean = grouped.values

dict = {'participID': participI, 'semester_compliance': mean}  
    
dfF = pd.DataFrame(dict)
df = df.merge(dfF,right_on = 'participID',left_on = 'participID')

In [11]:
df = df.merge(dfb,right_on = 'id',left_on = 'participID')

In [12]:
df['left_study'] = 0
df.loc[(df.date > s2Start)& (df.date < s2End),'left_study'] = 1


In [13]:
df = df.groupby('participID').tail(1)


In [14]:
df.left_study.value_counts()

0    515
1     99
Name: left_study, dtype: int64

In [15]:
df3 = df
df3 = df3.groupby('participID').tail(1)
df3 = df3[['week','semester_compliance','left_study']]
df3 = df3.dropna()
df2 =df
df2.Extraversion_2 = df2.Extraversion_2.fillna(df2.Extraversion_1)
df2.Agreeableness_2 = df2.Agreeableness_2.fillna(df2.Agreeableness_1)
df2.Openness_2 = df2.Openness_2.fillna(df2.Openness_1)
df2.Neuroticism_2 = df2.Neuroticism_2.fillna(df2.Neuroticism_1)
df2.Conscientiousness_2 = df2.Conscientiousness_2.fillna(df2.Conscientiousness_1)
df2 = df2.groupby('participID').tail(1)
df4 = df2[['week','semester_compliance','left_study','Extraversion_2','Agreeableness_2','Conscientiousness_2','Openness_2','Neuroticism_2','gender_1','race_1','parentincome_1']]
df2 = df2[['week','Tier','week_after_intervention','semester_compliance','left_study','Extraversion_2','Agreeableness_2','Conscientiousness_2','Openness_2','Neuroticism_2','gender_1','parentincome_1','race_1']]
df2 = df2.dropna()
df4 = df4.dropna()

In [16]:
df.shape

(614, 32)

In [ ]:
low = df.semester_compliance.quantile(.5)
df = df[df['week_after_intervention'] > 0]
dfLow= df[df['semester_compliance'] < low]

print(low)

In [ ]:
low.shape

In [35]:
dfLow.semester_compliance.mean()


51.1439674260005

In [36]:
print(low)

74.65714285714286


In [37]:
# grouped =dfLow.groupby('id')
# grouped = grouped['complyPercent'].mean()
# participI  =  grouped.index
# mean = grouped.values

# dict = {'id': participI, 'post_compliance': mean}  
# dfF = pd.DataFrame(dict)
# dfLow = dfLow.merge(dfF,right_on = 'id',left_on = 'participID')


In [38]:
dfLow.shape

(228, 32)

In [39]:
survivors = dfLow[dfLow['left_study'] == 0]
failers = dfLow[dfLow['left_study'] == 1]


In [73]:
l = [0,0,0,0,0]
for s in survivors.race_1:
    l[int(s)-1] = l[int(s)-1]+1
f = [0,0,0,0,0]
failers = failers.dropna()
for s in failers.race_1:
    f[int(s)-1] = f[int(s)-1]+1

In [74]:
print(l)
print(f)
print([l,f])

[10, 15, 15, 13, 91]
[4, 3, 5, 6, 28]
[[10, 15, 15, 13, 91], [4, 3, 5, 6, 28]]


In [75]:
from scipy.stats import chisquare

In [76]:

f_obs = np.array([l,f]).T

from scipy import stats
stats.chi2_contingency(f_obs)[0:3]

(1.3024512341168357, 0.8609595063035189, 4)

In [77]:
stats.chisquare(f_obs)

Power_divergenceResult(statistic=array([168.5       ,  48.56521739]), pvalue=array([2.19474611e-35, 7.19480322e-10]))

In [ ]:
!pip install FisherExact

In [ ]:
table = f_obs

In [ ]:
fisher_exact(table, alternative='two-sided', hybrid=False, midP=False, simulate_pval=False, replicate=2000, workspace=300, attempt=2, seed=None)

In [41]:
failers.Conscientiousness_1.mean()

3.6269841279302324

In [27]:
failC = list(failers.semester_compliance)
test_list = list(survivors.semester_compliance)
surviveC = []
for i in range(len(test_list)):
    if(test_list[i] > 0):
        surviveC.append(test_list[i])

print(len(surviveC))
print(len(failC))
print(failers.semester_compliance.mean())
print(survivors.semester_compliance.mean())

31
51
29.948765949189475
37.31502101174003


In [28]:

stats.ttest_ind(failC,surviveC,equal_var=False)



Ttest_indResult(statistic=-4.150587872109876, pvalue=8.292335119146005e-05)